In [3]:
%reload_ext autoreload
%autoreload 2

import csv
import numpy as np

In [6]:
with open('../../../ESB-Staging/ESB_CDWT_Setpoint.csv', 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        print (row)

['75.0']


In [5]:
with open('../../../ESB-Staging/ESB_CDWT_Setpoint.csv', 'w') as f:
    writer = csv.writer(f)
    v = np.array([75.0])
    writer.writerow(v)